<div class="admonition note">
<p class="admonition-title">Thin wrapper</p>
<p>Lightweight adapter around
<a href="https://causalml.readthedocs.io/">causalml</a>
<a href="https://causalml.readthedocs.io/en/latest/methodology.html#matching"><code>NearestNeighborMatch</code></a>.
All statistical logic is delegated to the library.</p>
</div>

# Neighbour Matching

This notebook demonstrates **nearest neighbour matching** impact estimation using `evaluate_impact()`.

The model matches treated and control units on observed covariates (price), then computes ATT (Average Treatment Effect on the Treated), ATC (on the Controls), and ATE (overall) from mean outcome differences in the matched sample.

## Workflow Overview

1. User provides `products.csv`
2. User configures `DATA.ENRICHMENT` for treatment assignment
3. User calls `evaluate_impact(config.yaml)`
4. Engine handles everything internally (adapter, enrichment, model)

## Initial Setup

In [ ]:
import json
from pathlib import Path

import pandas as pd
from impact_engine_measure import evaluate_impact, parse_config_file
from impact_engine_measure.metrics import create_metrics_manager
from impact_engine_measure.models.factory import get_model_adapter
from online_retail_simulator import simulate

## Step 1: Product Catalog

In production, this would be your actual product catalog.

In [ ]:
output_path = Path("output/demo_nearest_neighbour_matching")
output_path.mkdir(parents=True, exist_ok=True)

job_info = simulate("configs/demo_nearest_neighbour_matching_catalog.yaml", job_id="catalog")
products = job_info.load_df("products")

print(f"Generated {len(products)} products")
print(f"Products catalog: {job_info.get_store().full_path('products.csv')}")
products.head()

## Step 2: Engine Configuration

Configure the impact engine with:
- **ENRICHMENT**: Treatment assignment via quality boost (50/50 split)
- **MODEL**: `nearest_neighbour_matching` matching on `price`

Matching parameters:
- `caliper: 0.2` — maximum allowed distance between matched pairs
- `replace: true` — control units can be reused across matches
- `ratio: 1` — one-to-one matching

In [ ]:
config_path = "configs/demo_nearest_neighbour_matching.yaml"
baseline_config_path = "configs/demo_nearest_neighbour_matching_baseline.yaml"

## Step 3: Impact Evaluation

A single call to `evaluate_impact()` handles everything:
- Engine creates CatalogSimulatorAdapter
- Adapter simulates metrics (single-day, cross-sectional)
- Adapter applies enrichment (treatment assignment + revenue boost)
- NearestNeighbourMatchingAdapter matches on price, computes ATT/ATC/ATE

In [ ]:
results_path = evaluate_impact(config_path, str(output_path), job_id="results")
print(f"Results saved to: {results_path}")

## Step 4: Review Results

In [ ]:
with open(results_path) as f:
    results = json.load(f)

data = results["data"]
estimates = data["impact_estimates"]
summary = data["model_summary"]

print("=" * 60)
print("NEAREST NEIGHBOUR MATCHING RESULTS")
print("=" * 60)

print(f"\nModel Type: {results['model_type']}")

print("\n--- Impact Estimates ---")
print(f"ATT (Avg Treatment on Treated):  {estimates['att']:.4f} (SE: {estimates['att_se']:.4f})")
print(f"ATC (Avg Treatment on Controls): {estimates['atc']:.4f} (SE: {estimates['atc_se']:.4f})")
print(f"ATE (Avg Treatment Effect):      {estimates['ate']:.4f}")

print("\n--- Matching Summary ---")
print(f"Observations:    {summary['n_observations']}")
print(f"Treated:         {summary['n_treated']}")
print(f"Control:         {summary['n_control']}")
print(f"Matched (ATT):   {summary['n_matched_att']}")
print(f"Matched (ATC):   {summary['n_matched_atc']}")
print(f"Caliper:         {summary['caliper']}")
print(f"Replace:         {summary['replace']}")
print(f"Ratio:           {summary['ratio']}")

In [ ]:
# Covariate balance artifacts
results_dir = Path(results_path).parent

balance_before = pd.read_parquet(results_dir / "nearest_neighbour_matching__balance_before.parquet")
balance_after = pd.read_parquet(results_dir / "nearest_neighbour_matching__balance_after.parquet")

print("--- Covariate Balance Before Matching ---")
print(balance_before.to_string())
print("\n--- Covariate Balance After Matching ---")
print(balance_after.to_string())

print("\n" + "=" * 60)
print("Demo Complete!")
print("=" * 60)

## Step 5: Model Validation

Compare the model's ATT estimate against the **true causal effect** computed from counterfactual vs factual data.

In [ ]:
def calculate_true_effect(
    baseline_metrics: pd.DataFrame,
    enriched_metrics: pd.DataFrame,
) -> dict:
    """Calculate TRUE ATT by comparing per-product revenue for treated products."""
    treated_ids = enriched_metrics[enriched_metrics["enriched"]]["product_id"].unique()

    enriched_treated = enriched_metrics[enriched_metrics["product_id"].isin(treated_ids)]
    baseline_treated = baseline_metrics[baseline_metrics["product_id"].isin(treated_ids)]

    enriched_mean = enriched_treated.groupby("product_id")["revenue"].mean().mean()
    baseline_mean = baseline_treated.groupby("product_id")["revenue"].mean().mean()
    treatment_effect = enriched_mean - baseline_mean

    return {
        "enriched_mean": float(enriched_mean),
        "baseline_mean": float(baseline_mean),
        "treatment_effect": float(treatment_effect),
    }

In [ ]:
parsed_baseline = parse_config_file(baseline_config_path)
baseline_manager = create_metrics_manager(parsed_baseline)
baseline_metrics = baseline_manager.retrieve_metrics(products)

parsed_enriched = parse_config_file(config_path)
enriched_manager = create_metrics_manager(parsed_enriched)
enriched_metrics = enriched_manager.retrieve_metrics(products)

print(f"Baseline records: {len(baseline_metrics)}")
print(f"Enriched records: {len(enriched_metrics)}")

In [ ]:
true_effect = calculate_true_effect(baseline_metrics, enriched_metrics)

true_te = true_effect["treatment_effect"]
model_te = estimates["att"]

if true_te != 0:
    recovery_accuracy = (1 - abs(1 - model_te / true_te)) * 100
else:
    recovery_accuracy = 100 if model_te == 0 else 0

print("=" * 60)
print("TRUTH RECOVERY VALIDATION")
print("=" * 60)
print(f"True treatment effect:  {true_te:.4f}")
print(f"Model ATT estimate:     {model_te:.4f}")
print(f"Recovery accuracy:      {max(0, recovery_accuracy):.1f}%")
print("=" * 60)

### Convergence Analysis

How does the estimate converge to the true effect as sample size increases?

In [ ]:
sample_sizes = [20, 50, 100, 200, 300, 500, 1500]
estimates_list = []
truth_list = []

parsed = parse_config_file(config_path)
measurement_config = parsed["MEASUREMENT"]
all_product_ids = enriched_metrics["product_id"].unique()

for n in sample_sizes:
    subset_ids = all_product_ids[:n]
    enriched_sub = enriched_metrics[enriched_metrics["product_id"].isin(subset_ids)]
    baseline_sub = baseline_metrics[baseline_metrics["product_id"].isin(subset_ids)]

    true = calculate_true_effect(baseline_sub, enriched_sub)
    truth_list.append(true["treatment_effect"])

    model = get_model_adapter("nearest_neighbour_matching")
    model.connect(measurement_config["PARAMS"])
    result = model.fit(data=enriched_sub, dependent_variable="revenue")
    estimates_list.append(result.data["impact_estimates"]["att"])

print("Convergence analysis complete.")

In [ ]:
from notebook_support import plot_convergence

plot_convergence(
    sample_sizes,
    estimates_list,
    truth_list,
    xlabel="Number of Products",
    ylabel="Treatment Effect (ATT)",
    title="Nearest Neighbour Matching: Convergence of Estimate to True Effect",
)